# Creating interactive maps with Leaflet and Folium 

### Install updates and dependencies for Earth Engine API and Python Client

This is a one time step.

We are going to update Conda, and install a few requirements for running geospatial data in the notebook.

In [ ]:
%%bash
conda update -n base conda &&
conda install -y -c conda-forge google-api-python-client earthengine-api  folium nodejs cython gdal

In [ ]:
%%bash
pip install --upgrade pip &&
pip install vega_datasets &&
pip install pyCrypto &&
pip install kml2geojson

In [14]:
%%bash
jupyter labextension install @jupyterlab/geojson-extension

jupyterlab-geojson-extension-0.16.0.tgz
yarn install v1.3.2
info No lockfile found.
[1/4] Resolving packages...
[2/4] Fetching packages...
info fsevents@1.2.4: The platform "linux" is incompatible with this module.
info "fsevents@1.2.4" is an optional dependency and failed compatibility check. Excluding it from installation.
[3/4] Linking dependencies...
warning "@jupyterlab/json-extension > react-json-tree@0.10.9" has incorrect peer dependency "react@^15.0.0".
warning "@jupyterlab/json-extension > react-json-tree@0.10.9" has incorrect peer dependency "react-dom@^15.0.0".
warning "@jupyterlab/vdom-extension > @nteract/transform-vdom@1.1.1" has unmet peer dependency "react@^15.6.1".
[4/4] Building fresh packages...
warning Error running install script for optional dependency: "/opt/anaconda3/share/jupyter/lab/staging/node_modules/canvas: Command failed.\nExit code: 1\nCommand: node-gyp rebuild\nArguments: \nDirectory: /opt/anaconda3/share/jupyter/lab/staging/node_modules/canvas\nOutput:

> /home/tyson_swetnam/.conda/envs/ipykernel_py2/bin/npm pack @jupyterlab/geojson-extension
> node /opt/anaconda3/lib/python3.6/site-packages/jupyterlab/staging/yarn.js install
> node /opt/anaconda3/lib/python3.6/site-packages/jupyterlab/staging/yarn.js run build:prod


## Authenticating to the Earth Engine servers

In [1]:
%%bash
earthengine authenticate --quiet

bash: line 1: earthengine: command not found


Once you have obtained an authorization code from the previous step, paste the code into the following cell and run it:

In [12]:
%%bash
earthengine authenticate --authorization-code=4/AAAet1gvcEYuOaaIFBMrxqWuGj27hco0goBFhyWm7p8dJxAn6wvgvt4


Successfully saved authorization token.


## Import Python packages

In [18]:
# import packages
import pandas as pd
import json
import sys
import earthengine as ee

ee.Initialize()

import folium
print(folium.__version__)
sys.path.append('..')
print (folium.__file__)
print (folium.__version__)

ModuleNotFoundError: No module named 'earthengine'

### Create definition files for working with Google Earth Engine

In [4]:
def folium_gee_map(image,vis_params=None,folium_kwargs={}):
    """
    Function to view Google Earth Engine tile layer as a Folium map.
    
    Parameters
    ----------
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    return folium.Map(**folium_kwargs)

def folium_gee_layer(folium_map,image,vis_params=None,folium_kwargs={}):
    """
    Function to add Google Earch Engine tile layer as a Folium layer.
    
    Parameters
    ----------
    folium_map : Folium map to add tile to.
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    layer = folium.TileLayer(**folium_kwargs)
    layer.add_to(folium_map)


### Initiate your connection to the CyVerse DataStore with iRODS iCommands

This little script _MUST_ be run on a secure connection, else your password will be visible to anyone monitoring your work.

If you're not comfortable entering your CyVerse password in the notebook, you can open a termianl session and type 

```
iinit
```

You will be asked for your password there.

In [10]:
import getpass
import os

_password = getpass.getpass()
command = "iinit" # can be any command but don't forget -S as it enables input from stdin
os.popen(command, 'w').write(_password+'\n') # newline char is important otherwise prompt will wait for you to manually perform newline
del(_password) # deletes your password on the notebook

Now, we want to copy our data to a new scratch directory. If your user does not own the directory you will have to also use `chmod` to change ownership.

```
sudo mkdir -p /scratch/2016_Campaign/HARV/L1/DiscreteLidar
sudo chown $USER:iplant-everone /scratch/ -R
```

Note, you will have to use `sudo` for directories your username does not own.

In [14]:
import getpass
import os
_password = getpass.getpass()
command = "sudo chown $USER:$iplant-everyone /scratch/ -R" # can be any command but don't forget -S as it enables input from stdin
os.popen(command, 'w').write(_password+'\n') # newline char is important otherwise prompt will wait for you to manually perform newline
del(_password) # deletes your password on the notebook

In [21]:
!mkdir -p /scratch/2016_Campaign/HARV/L1/DiscreteLidar

total 100K
drwxr-xr-x  23 root          root            4.0K May  2 11:38 .
drwxr-xr-x  23 root          root            4.0K May  2 11:38 ..
drwxr-xr-x   2 root          root            4.0K Apr 27 15:16 bin
drwxr-xr-x   4 root          root            4.0K Apr 27 15:23 boot
drwxr-xr-x  17 root          root            3.6K May  2 11:34 dev
drwxr-xr-x 124 root          root             12K May  2 13:07 etc
drwxr-xr-x   3 root          root            4.0K May  2 11:36 home
lrwxrwxrwx   1 root          root              33 Apr 25 22:11 initrd.img -> boot/initrd.img-4.15.0-20-generic
lrwxrwxrwx   1 root          root              33 Apr 25 22:11 initrd.img.old -> boot/initrd.img-4.15.0-20-generic
drwxr-xr-x  20 root          root            4.0K Apr 27 15:16 lib
drwxr-xr-x   2 root          root            4.0K Apr 25 22:10 lib64
drwx------   2 root          root             16K Apr 25 22:14 lost+found
drwxr-xr-x   2 root          root            4.0K Apr 25 22:10 media
-rw-r--r--   1 r

In [1]:
!iget -KPQbrv /iplant/home/shared/NEON_data_institute_2018/2016_Campaign/HARV/L1/DiscreteLidar /scratch/2016_Campaign/HARV/L1/DiscreteLidar

D- /scratch/2016_Campaign/HARV/L1/DiscreteLidar/DiscreteLidar :
D- /scratch/2016_Campaign/HARV/L1/DiscreteLidar/DiscreteLidar/ClassifiedLaz :
0/4957 -  0.00% of files done   0.000/46530.193 MB -  0.00% of file sizes done
Processing 2016_HARV_1_722000_4703000.laz - 0.141 MB   2018-05-03.10:35:55
ERROR: getCollUtil: getDataObjUtil failed for /iplant/home/shared/NEON_data_institute_2018/2016_Campaign/HARV/L1/DiscreteLidar/ClassifiedLaz/2016_HARV_1_722000_4703000.laz. status = -312000 status = -312000 OVERWRITE_WITHOUT_FORCE_FLAG
0/4957 -  0.00% of files done   0.000/46530.193 MB -  0.00% of file sizes done
Processing 2016_HARV_1_722000_4705000.laz - 0.000 MB   2018-05-03.10:35:55
ERROR: getCollUtil: getDataObjUtil failed for /iplant/home/shared/NEON_data_institute_2018/2016_Campaign/HARV/L1/DiscreteLidar/ClassifiedLaz/2016_HARV_1_722000_4705000.laz. status = -312000 status = -312000 OVERWRITE_WITHOUT_FORCE_FLAG
0/4957 -  0.00% of files done   0.000/46530.193 MB -  0.00% of file sizes done

In [30]:
import kml2geojson
kml2geojson.main.convert('/scratch/2016_Campaign/HARV/L1/DiscreteLidar/LasBoundary/full_boundary.kml', '/scratch/2016_Campaign/HARV/L1/DiscreteLidar/LasBoundary/')

In [31]:
# import geo json data
import json
full_boundary = json.load(open('/scratch/2016_Campaign/HARV/L1/DiscreteLidar/LasBoundary/full_boundary.geojson'))

In [32]:
# Get an area to look at
lat = 42.500
lon = -72.165
zoom_start=11

# Open Street Map Base
m = folium.Map(location=[lat, lon], tiles="OpenStreetMap", zoom_start=zoom_start)

# Add polygon outline of HARV
full_boundary = json.load(open('/scratch/2016_Campaign/HARV/L1/DiscreteLidar/LasBoundary/full_boundary.geojson'))
folium.GeoJson(full_boundary).add_to(m)

# Add GEE Terrain Layer
image = ee.Image('srtm90_v4')
vis_params = {'min':0.0, 'max':3000, 'palette':'00FFFF,0000FF'}
folium_gee_layer(m,image,vis_params=vis_params,folium_kwargs={'overlay':True,'name':'SRTM'})

# Create a reference to the LANDSAT 8 image collection
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')
# Filter the LANDSAT collection down to a eight month period
filtered = l8.filterDate('2016-05-01', '2016-09-30');
# Use the mosaic reducer, to select the most recent pixel in areas of overlap
l8_image = filtered.median()
l8_vis_params = {'min': 0, 'max':0.3, 'bands':'B4,B3,B2'}
folium_gee_layer(m,l8_image,l8_vis_params,folium_kwargs={'overlay':True,'name':'LANDSAT'})

# Create a reference to the 2017 SRER NEON orthophotography image collection
#neon = ee.ImageCollection('users/gponce/usda_ars/image_collections/neon_srer_2017_rgb')
#neon_image = neon.median()
#n eon_vis_params = {'min':0, 'max':255, 'bands':'b1,b2,b3'}
#folium_gee_layer(m,neon_image,neon_vis_params,folium_kwargs={'overlay':True,'name':'NEON'})

# Create a reference to the 2016 sUAS orthophotography imager collection

m.add_child(folium.LayerControl())
m

In [8]:
import gdal as gdal
import osr as osr

gdal.UseExceptions()

fname = '/home/tswetnam/wgew/LH_20agl_9Oct2017_georef.tif'

ds = gdal.Open(fname)
data = ds.ReadAsArray()
gt = ds.GetGeoTransform()
proj = ds.GetProjection()

inproj = osr.SpatialReference()
inproj.ImportFromWkt(proj)

print(inproj)

ModuleNotFoundError: No module named 'gdal'

In [6]:
!iget -KPQvrf /iplant/home/shared/srer-wgew/data/wgew/sfm_2017/LH_20agl_9Oct2017_georef.tif

0/1 -  0.00% of files done   0.000/6512.374 MB -  0.00% of file sizes done
Processing LH_20agl_9Oct2017_georef.tif - 6512.374 MB   2018-04-19.13:22:11
LH_20agl_9Oct2017_georef.tif - 840.000/6512.374 MB - 12.90% done   2018-04-19.13:22:23
LH_20agl_9Oct2017_georef.tif - 1880.000/6512.374 MB - 28.87% done   2018-04-19.13:22:32
LH_20agl_9Oct2017_georef.tif - 2680.000/6512.374 MB - 41.15% done   2018-04-19.13:22:39
LH_20agl_9Oct2017_georef.tif - 3520.000/6512.374 MB - 54.05% done   2018-04-19.13:22:46
LH_20agl_9Oct2017_georef.tif - 4080.000/6512.374 MB - 62.65% done   2018-04-19.13:22:53
LH_20agl_9Oct2017_georef.tif - 4927.023/6512.374 MB - 75.66% done   2018-04-19.13:22:59
LH_20agl_9Oct2017_georef.tif - 5595.117/6512.374 MB - 85.92% done   2018-04-19.13:23:05
LH_20agl_9Oct2017_georef.tif - 6230.234/6512.374 MB - 95.67% done   2018-04-19.13:23:10
LH_20agl_9Oct2017_georef.tif - 6465.351/6512.374 MB - 99.28% done   2018-04-19.13:23:11
LH_20agl_9Oct2017_georef.tif - 6505.351/6512.374 MB - 99.8

In [43]:
!gdalwarp -t_srs EPSG:4326 -r near /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.tif /home/tswetnam/wgew/LH_20agl_9Oct2017_georef2.tif

Processing input file /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.tif.
Using internal nodata values (e.g. 256) for image /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.tif.
ERROR 1: Too many points (441 out of 441) failed to transform, unable to compute output bounds.
Warning 1: Unable to compute source region for output window 0,0,36805,31362, skipping.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [44]:
!gdal_translate -of mbtiles  /home/tswetnam/wgew/LH_20agl_9Oct2017_georef2.tif  /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.mbtiles
!gdaladdo -r nearest /home/tswetnam/wgew/LH_20agl_9Oct2017_georef.mbtiles 2 4 8 16

Input file size is 36805, 31362
ERROR 6: zoom_level > 22 not supported
0...10...20...30...40...50...60...70...80...90...100 - done.
ERROR 6: zoom_level > 22 not supported
